In [1]:
import pandas as pd
import os
import requests

In [2]:
raidDir   = '/Users/amcguire/Documents/PERSONAL/KOL/raidLogs/'
newRDir   = '/Users/amcguire/Library/Application Support/KoLmafia/data/test/'
outputDir = '/Users/amcguire/Documents/PERSONAL/KOL/'

# users who are running dread w/ alts
multiUser = {'madowl (combined)':['themadowl','madowl','Arkcon']}

# clan maps. keeping HoD in here even tho i took it out of my clanHop.ash
clanMap   = {'The Hogs of Exploitery':'HoE', 
             'KirbyLlama\'s Private Dungeon Clan': 'KirbyClan',
             'The Piglets of Fate': 'Piglets',
             'The Hogs of Destiny': 'HoD',
             'The 100% of Scientific FACT Club': 'FactClub',
             'Castle Greyhawk': 'Greyhawk'}

# folks who have all their skills and thus are outta the skill ladder
finishedUsers = ['violetinsane','Captain Scotch','kenny kamAKAzi','The Dictator']

# run files from my old EXTREMELY manual process
runFiles  = ['r1-3389','r2-3869','r3-4283','r4-4190','r5-3503','r6-3598',
             'r7-4372','r8-3473','r9-3858','r10-3787','r11-3797','r12-3798',
             'r13-12260','r14-8393','r15-3769','r16-3496','r17-4041','r18-3582',
             'r19-4318','r20-3584','r21-3794','r22-4698','r25-3802','r69-1337']

In [3]:
# Temporary script to convert Stary script output into old c/p format

for file in os.listdir(newRDir): # i wish listdir could give datetime info
    print(file)
    try:    
        clan = file.split(sep='_')[1]
        rNum = file.split(sep='_')[2][:6]
        newFileName = 'r{1}_{0}.txt'.format(clanMap[clan],rNum)

        currFile = open('{}/{}'.format(newRDir, file)).read()

        newFile = ''

        for ct, line in enumerate(currFile.split(sep='\n')):
            try:     newFile = newFile + line.split(sep='\t')[1].replace('<b>','').replace('</b>','') + '\n'
            except:  newFile = newFile + 'LINE-ERROR' + '\n'

        print('  --- > {} file parsed. {} new lines.'.format(clan,ct))

        # it is kind of dumb that i just keep writing to my old raidLog directory 
        #   in old raidLog format but i mean, the code works, so why mess it up
        #   too much i guess ???
        file = open('{}{}'.format(raidDir,newFileName),"w")

        file.write(newFile)
        file.close()

        runFiles = runFiles + [newFileName[:-4]]
    except:
        print('  --- !!!!!!!!!!!!!!!! stupid thing fucked up !!!!!!!!!!!!!!!!')

dungeon_The Hogs of Exploitery_189472.txt
  --- > The Hogs of Exploitery file parsed. 115 new lines.
.DS_Store
  --- !!!!!!!!!!!!!!!! stupid thing fucked up !!!!!!!!!!!!!!!!
dungeon_The Hogs of Exploitery_188769.txt
  --- > The Hogs of Exploitery file parsed. 184 new lines.
dungeon_KirbyLlama's Private Dungeon Clan_188757.txt
  --- > KirbyLlama's Private Dungeon Clan file parsed. 189 new lines.
dungeon_The Hogs of Destiny_188819.txt
  --- > The Hogs of Destiny file parsed. 189 new lines.
dungeon_Castle Greyhawk_188871.txt
  --- > Castle Greyhawk file parsed. 137 new lines.
dungeon_The Piglets of Fate_188579.txt
  --- > The Piglets of Fate file parsed. 283 new lines.
dungeon_The Hogs of Destiny_188572.txt
  --- > The Hogs of Destiny file parsed. 139 new lines.
dungeon_The Piglets of Fate_189481.txt
  --- > The Piglets of Fate file parsed. 167 new lines.
dungeon_The Hogs of Exploitery_189236.txt
  --- > The Hogs of Exploitery file parsed. 166 new lines.
dungeon_The Piglets of Fate_189252

In [4]:
# Populate our run dictionary
allRuns = ""

for run, txtLog in enumerate(runFiles):
    # Python is zero-indexed, so we increment run by 1 here. 
    #   Also replace spaces w/ _ for playername.
    
    allRuns = allRuns+open('{}{}.txt'.format(raidDir, txtLog)).read()+"\n"

In [5]:
def collectRaiders(runs):
    """ Function to collect all dungeon raiders. Only grabs folks who
        have defeated at least one monster across all ASS runs. """
    
    raiders = []
    
    for row in runs.split('\n'):
        if ('defeated' in row) & (' (#' in row):
            name = row[:row.find(' (#')]
            raiders = list(set(raiders + [name]))
    
    return raiders
  
# ... why did I make this function, anyway?
# collectRaiders(allRuns)

In [6]:
def tabulateKills(name, runs):
    """ Function to tabulate how many kills a specific individual had.
        Also outputs defeats, if we actually want that. """
    
    kills  = 0
    killed = 0
    
    for row in runs.split('\n'):
        if (name in row) & ('defeated' in row):
            if row[0:len(name)] == name: # fixing the madowl bug ;-;
                if 'was defeated' in row:
                    tSpent = int(row.split(' (')[2].split(' ')[0])
                    killed += tSpent
                else:
                    tSpent = int(row.split(' (')[2].split(' ')[0])
                    kills  += tSpent
                
    return({'kills':kills,'defeats':killed})

In [7]:
def tabulateLoots(name, runs):
    ''' Function to tabulate loot per individuals participating. Also 
        grabs skills, just so that we have em.'''
    
    loot   = []
    skills = 0
    
    for row in runs.split('\n'):
        if ('distributed' in row) and (name in row):
            distTo = row.split(' to ')[1].split(' (#')[0]
            if distTo == name:
                thisLoot = row.split(' distributed ')[1].split(' to ')[0]
                loot = loot + [thisLoot]
        elif (' used The Machine' in row) and (name in row):
            distTo = row.split(' used The ')[0].split(' (#')[0]
            if distTo == name: skills+=1
        
    # Parse out things we don't want to be considered as "true" loot
    notRealLoot = ['ghost pepper','electric Kool-Aid','skull capacitor',
                   'wriggling severed nose','Hunger™ Sauce','bottle of Bloodweiser']
    finalLoot = [x for x in loot if x not in notRealLoot]
    
    return({'loot':finalLoot,'skills':skills})

# unfortunately stary's clanHop script doesn't actually save loot. so, basically all loot is sourced from my manual logs  
# tabulateLoots('Captain Scotch', allRuns)

In [8]:
finalDisplay = {}

# Create summary dictionary via tabulation code

# ... oh, THAT'S why i made collectRaiders, yes, this makes sense
for name in collectRaiders(allRuns):
    
    finalDisplay[name] = {}
    
    finalDisplay[name].update(tabulateKills(name, allRuns))
    finalDisplay[name].update(tabulateLoots(name, allRuns))
    finalDisplay[name]['finishedFlag'] = [1 if name in finishedUsers else 0][0]
    
    finalDisplay[name]['lootCount'] = len(finalDisplay[name]['loot'])

In [9]:
# Slice of code for combining duplicate users

for name in multiUser.keys():
    
    finalDisplay[name] = {}
    
    for field in ['kills','defeats','loot','skills','lootCount']:
        try:    finalDisplay[name][field] = sum([finalDisplay[x][field] for x in multiUser[name]])
        except: finalDisplay[name][field] = [].append([finalDisplay[x][field] for x in multiUser[name]])
            
    for x in multiUser[name]: finalDisplay.pop(x)

In [10]:
finalFrame = pd.DataFrame.from_dict(finalDisplay,orient='index').sort_values(by='kills',ascending=False).fillna(0)
finalFrame = finalFrame.loc[:,['kills','defeats','skills','lootCount','loot','finishedFlag']]
finalFrame['kill/skillz'] = finalFrame['kills']/(finalFrame['skills']+0.5)

finalFrame = finalFrame.sort_values(['finishedFlag','kill/skillz'], ascending=[True,False])
finalFrame

,kills,defeats,skills,lootCount,loot,finishedFlag,kill/skillz
madowl (combined),15634,738,13,21,0,0.0,1158.074074
RandomExtremity,7471,43,6,7,"[Drunkula's bell, Great Wolf's rocket launcher...",0.0,1149.384615
gregmasta,13927,25,12,12,"[Gets-You-Drunk, Unkillable Skeleton's shingua...",0.0,1114.160000
Liora,548,7,0,1,[Great Wolf's left paw],0.0,1096.000000
Stary,6674,72,6,7,"[Mayor Ghost's scissors, Drunkula's cape, Grea...",0.0,1026.769231
Name Guy Man,7659,7,7,17,"[Quiets-Your-Steps, Covers-Your-Head, Drunkula...",0.0,1021.200000
Just Eyes,7521,94,7,7,"[Gets-You-Drunk, Drunkula's wineglass, Protect...",0.0,1002.800000
Whym,5368,1,5,1,[Gets-You-Drunk],0.0,976.000000
Phillammon,5313,74,5,15,"[Drunkula's bell, Gets-You-Drunk, HOA regulati...",0.0,966.000000
chameco,4197,0,4,1,[Gets-You-Drunk],0.0,932.666667


In [11]:
print("This effort has gotten {} skills for {} Loathers. Good job folks!".format(finalFrame['skills'].sum(),len(finalFrame['skills'])))

This effort has gotten 125 skills for 27 Loathers. Good job folks!


In [12]:
from datetime import date

currDate = str(date.today().strftime('%m%d'))
finalFrame.to_csv('{}dungeonLogs{}.csv'.format(outputDir,currDate))

In [13]:
def lineParse(string):
    ''' The buffer bit is when I was erroneously thinking that defeats somehow 
        counted against us. They do not. Still, left the logic in just in case
        I want to make this more complicated in the future, and get differential
        information for different parsing situations.'''
    
    if 'was defeated' in string:
        return {'buffer':int(string.split(' (')[2].split(' ')[0]),'kills':0}
    else:
        try:
            if ') defeated' not in string: print(string)
            return {'kills':int(string.split(' (')[2].split(' ')[0]),'buffer':0}
        except: 
            # print out when this encounters an error; this is how i found all the exceptions below
            print(string)
            return {'buffer':0,'kills':0}

def tabulateSummary(runs, printFlag = True):
    """ Function to tabulate a general summary of a run. """
    
    # i am 100000% positive there is a better way to do this, or to mass-assign
    #   null variables. i cannot for the life of me remember what it is.
    
    woodBossKilled = 0
    villBossKilled = 0
    castBossKilled = 0
    
    bugbear  = 0
    werewolf = 0
    
    zombie   = 0
    ghost    = 0
    
    skeleton = 0
    vampire  = 0
    
    machine = 0
    
    # some loot/actions mess w/ the string parsing here; manually exclude em
    exclTable = ['ghost pepper', 'HOA zombie eyes', 'zombie accordion', 
                 'zombie mariachi pants', 'zombie mariachi hat', 
                 'drove some zombies out', 'ghost pencil', 'ghost shawl',
                 'drove some skeletons out', 'drove some vampires']
    
    # this isn't particularly elegant but it's fast
    for row in runs.split('\n'):
        if any(excl in row for excl in exclTable):
            continue
        elif ('bugbear'  in row):
            bugbear  += lineParse(row)['kills']
        elif ('werewolf' in row):
            werewolf  += lineParse(row)['kills']
        elif ('zombie'   in row):
            zombie  += lineParse(row)['kills']
        elif ('ghost'    in row):
            ghost  += lineParse(row)['kills']
        elif ('skeleton' in row):
            skeleton  += lineParse(row)['kills']
        elif ('vampire'  in row):
            vampire  += lineParse(row)['kills']
        elif ('used The Machine' in row):
            machine += 1
            
        # bosses; this is currently unnecessary but i was curious how many 
        #   bosskillings i actually captured, since i'm not the only one who
        #   cycles instances and i don't always snag an update beforehand
        
        elif ('  Falls-From-Sky' in row):
            woodBossKilled += lineParse(row)['kills']
        elif ('The Great Wolf of the Air ' in row):
            woodBossKilled += lineParse(row)['kills']
        elif ('  Mayor Ghost' in row):
            villBossKilled += lineParse(row)['kills']
        elif (' the Zombie Homeowners\' Association ' in row):
            villBossKilled += lineParse(row)['kills']
        elif ('  Count Drunkula ' in row):
            castBossKilled += lineParse(row)['kills']
        elif ('  The Unkillable Skeleton' in row):
            castBossKilled += lineParse(row)['kills']
    
    # was bugchecking a stupid error
    # print("{} {} | {} {} | {} {}".format(bugbear,werewolf,zombie,ghost,vampire,skeleton))
    
    woodPct = bugbear/(bugbear+werewolf+0.00001)
    villPct = zombie/(zombie+ghost+0.00001)
    castPct = vampire/(vampire+skeleton+0.00001)
    
    # my initial thought was that anything over 50% guaranteed that boss; this is 
    #   not strictly true apparently, and you can occasionally get flips w/ <60%. 
    #   so this checks so that we have a clear sense of what the exact diff is
    
    if woodPct>0.5: woodBoss = 'FFS ({:0.2f})'.format(woodPct)      # there is a % format but yolo
    else:           woodBoss = 'Wolf ({:0.2f})'.format(1-woodPct)
        
    if villPct>0.5: villBoss = 'ZHoA ({:0.2f})'.format(villPct)
    else:           villBoss = 'MG ({:0.2f})'.format(1-villPct)
        
    if castPct>0.5: castBoss = 'Drunkula ({:0.2f})'.format(castPct)
    else:           castBoss = 'UKS ({:0.2f})'.format(1-castPct)
    
    # (the stupid error was that the castle was "skeleton - zombie" so i had a bunch of weird results)
    if printFlag == True:   
        print('==== RUN SUMMARY ==== \n   KILLS LEFT: {}/{}/{}\n   BOSSES: {}, {}, {}\n   MACHINE: {} skills left!\n   BOSS DEFEATS: {} {} {}'.format(
        1000-(bugbear+werewolf),1000-(ghost+zombie),1000-(skeleton+vampire),woodBoss,villBoss,castBoss,3-machine,woodBossKilled, villBossKilled, castBossKilled))
    
    # return is for the text update creator and naught else
    return '{}/{}/{} ({} skills left! -- {}, {}, {})'.format(
        1000-(bugbear+werewolf),1000-(ghost+zombie),1000-(skeleton+vampire),3-machine,woodBoss,villBoss,castBoss)
    

In [14]:
# commented out because it's quite long. but if you want to summarize everything, this is it!

# for run in runFiles:
#     print('=====================\n---- {}'.format(run))
#     tabulateSummary(open('{}{}.txt'.format(raidDir, run)).read()+"\n")

# also, while i'm commenting stuff out, this is a snippet to get a player's involvement in all runs

# n = "The Dictator"
# for run in runFiles:
#     print('{} - {}'.format(run,str(tabulateKills(n, open('{}{}.txt'.format(raidDir, run)).read()+"\n"))))

In [15]:
def buildTextUpdate(numInst = 5, rDir = newRDir,desc = 'yolo yolo yolo'):
    ''' Grabs the five most recent instances & generates info '''
    
    # most recent files using some garbo processing. there is inevitably a better
    #   way to access/read this, but i am on a plane with broken wifi, and this 
    #   does technically work on unix systems!
    fileTree = os.popen('ls -lrst "{}"'.format(rDir)).read().split('\n')[-1*(numInst+1):]
    
    infoDump = {}
    orderedL = []
    
    for fileRaw in fileTree:
        # unix system generates this in fixed width; the "53" buffer here is system-dependent
        if len(fileRaw) > 53: # have to have this to error-catch widowed newlines
            file = fileRaw[53:]
            clan = clanMap[file.split(sep='_')[1]]
            
            # get a summary for this specific recent clan
            infoDump[clan] = tabulateSummary(open('{}{}'.format(rDir, file)).read()+"\n",printFlag=False)
    
    # making an ordered list of ordered tuples w/ total kills per clan to order the update 
    for clan in infoDump:
        kLeft = sum([int(kills) for kills in infoDump[clan].split(' ')[0].split('/')])
        orderedL = orderedL + [(kLeft,clan)]
    
    # sort is an inplace operator in python
    orderedL.sort()
    
    clanUpdate = ''
    
    # instead of reversing the sort order i just append on the front. is that dumb? yeah. probably.
    for x in orderedL:
        clanUpdate = '**{}:** '.format(x[1]) + infoDump[x[1]] + '\n' + clanUpdate
    
    # print that dang update buddy
    print('** {} ASS DREAD UPDATE **\n\nhey @Dungeon Runners ! {}\n\n{}\nand your skill ladder!\n'.format(currDate,desc,clanUpdate))
    
    # return that dict i built just in case i use it for something else later i guess
    return infoDump

In [16]:
# just update the description and it'll generate a paste-able update! whoo!

description = "here is your update! hoe is ready to cycle, and factclub/piglets are getting there. go get your skills! also, welcome liora!"

buildTextUpdate(5,newRDir, desc = description)

** 0509 ASS DREAD UPDATE **

hey @Dungeon Runners ! here is your update! hoe is ready to cycle, and factclub/piglets are getting there. go get your skills! also, welcome liora!

**Piglets:** 1000/1000/1000 (3 skills left! -- Wolf (1.00), MG (1.00), UKS (1.00))
**HoE:** 10/10/10 (2 skills left! -- FFS (0.62), MG (0.59), UKS (0.60))
**Greyhawk:** 8/10/10 (0 skills left! -- FFS (0.60), ZHoA (0.60), Drunkula (0.60))
**FactClub:** 10/7/9 (3 skills left! -- Wolf (0.59), ZHoA (0.59), Drunkula (0.62))
**KirbyClan:** 10/0/10 (0 skills left! -- FFS (0.61), MG (0.59), UKS (0.61))

and your skill ladder!



{'HoE': '10/10/10 (2 skills left! -- FFS (0.62), MG (0.59), UKS (0.60))',
 'KirbyClan': '10/0/10 (0 skills left! -- FFS (0.61), MG (0.59), UKS (0.61))',
 'Piglets': '1000/1000/1000 (3 skills left! -- Wolf (1.00), MG (1.00), UKS (1.00))',
 'Greyhawk': '8/10/10 (0 skills left! -- FFS (0.60), ZHoA (0.60), Drunkula (0.60))',
 'FactClub': '10/7/9 (3 skills left! -- Wolf (0.59), ZHoA (0.59), Drunkula (0.62))'}